In [1]:
import os
#os.getcwd()
os.chdir("/Users/JidaThada/Desktop/2018 Fall/Machine Learning/Project/google_analytics")

In [2]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics
from sklearn.feature_selection import f_classif
#import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

## read file

In [3]:
processed_train_df = pd.read_csv("processed_train_df.csv", dtype={'fullVisitorId': 'str'})
processed_test_df = pd.read_csv("processed_test_df.csv", dtype={'fullVisitorId': 'str'})

In [4]:
processed_train_df = processed_train_df.drop('Unnamed: 0', axis=1)
processed_test_df = processed_test_df.drop('Unnamed: 0', axis=1)

### 5-fold

In [ ]:
import random

In [14]:
unique_visitorId = processed_train_df['fullVisitorId'].unique()
random.seed(123)
random.shuffle(unique_visitorId)
no_cust = len(unique_visitorId)
print(no_cust)

714167


In [16]:
fold = 5
id_cv = []
for i in range(fold):
    if i<fold-1:
        cur_cv = unique_visitorId[i*(no_cust//5):(i+1)*(no_cust//5)]
    else:
        cur_cv = unique_visitorId[i*(no_cust//5):no_cust]
    id_cv.append(cur_cv)   

### Run function for MSE and Testset prediction

In [17]:
def getMse(x_tr, train, val, log_y_tr_pred, log_y_val_pred):
    revenue = np.exp(log_y_tr_pred) - 1
    id_list = list(train['fullVisitorId'])
    
    d = {'fullVisitorId':id_list, 'PredictedRevenue':revenue}
    submit = pd.DataFrame(data=d)
    col = ['fullVisitorId', 'PredictedRevenue']
    submit = submit[col]
    submit = pd.DataFrame(submit.groupby('fullVisitorId')["PredictedRevenue"].sum().reset_index())
    
    
    submit['PredictedLogRevenue'] = np.log1p(submit['PredictedRevenue'])
    y_tr_pred = list(submit['PredictedLogRevenue'])
    
    y_train_sumrev = pd.DataFrame(train.groupby('fullVisitorId')["totals.transactionRevenue"].sum().reset_index())
    y_train_sumrev['totals.transactionRevenue'] = np.log1p(y_train_sumrev['totals.transactionRevenue'])
    y_tr = list(y_train_sumrev['totals.transactionRevenue'])
    
    mse_tr = mean_squared_error(y_tr, y_tr_pred)
    print('train_mse', mse_tr)
    print('train_rmse', np.sqrt(mse_tr))

    
    revenue = np.exp(log_y_val_pred) - 1
    id_list = list(val['fullVisitorId'])
    
    d = {'fullVisitorId':id_list, 'PredictedRevenue':revenue}
    submit = pd.DataFrame(data=d)
    col = ['fullVisitorId', 'PredictedRevenue']
    submit = submit[col]
    submit = pd.DataFrame(submit.groupby('fullVisitorId')["PredictedRevenue"].sum().reset_index())
    submit['PredictedLogRevenue'] = np.log1p(submit['PredictedRevenue'])
    y_val_pred = list(submit['PredictedLogRevenue'])
    
    y_val_sumrev = pd.DataFrame(val.groupby('fullVisitorId')["totals.transactionRevenue"].sum().reset_index())
    y_val_sumrev['totals.transactionRevenue'] = np.log1p(y_val_sumrev['totals.transactionRevenue'])
    y_val = list(y_val_sumrev['totals.transactionRevenue'])
    
    mse_val = mean_squared_error(y_val, y_val_pred)
    
    print('val_mse', mse_val)
    print('val_rmse', np.sqrt(mse_val))
    return mse_tr, mse_val



In [26]:
def testset_pred(model, filename):
    x_tr = processed_train_df.iloc[:,2:]
    y_tr = processed_train_df.iloc[:,1]
    log_y_tr = np.log1p(y_tr)
    model.fit(x_tr, log_y_tr)
    
    x_te = processed_test_df.iloc[:,1:]
    te_id = processed_test_df.iloc[:,0]
    y_te_pred = model.predict(x_te)
    y_te_pred =[0 if i < 0 else i for i in y_te_pred]
    revenue = np.exp(y_te_pred) - 1
    
    # Write submit file
    d = {'fullVisitorId':te_id, 'PredictedLogRevenue':revenue}
    submit = pd.DataFrame(data=d)
    col = ['fullVisitorId', 'PredictedLogRevenue']
    submit = submit[col]
    submit = pd.DataFrame(submit.groupby('fullVisitorId')["PredictedLogRevenue"].sum().reset_index())
    submit['PredictedLogRevenue'] = np.log1p(submit['PredictedLogRevenue'])
    print(submit.shape)
    submit.head()
    submit.to_csv(filename+'.csv', index=False)


### Linear regression

In [19]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [20]:
train_mse = []
train_rmse = []
val_mse = []
val_rmse = []

for i in range(fold):
    print('\n\nfold:', i)
    val = processed_train_df[processed_train_df['fullVisitorId'].isin(id_cv[i])]
    train = processed_train_df[~processed_train_df['fullVisitorId'].isin(id_cv[i])]
    x_tr = train.iloc[:,2:]
    y_tr = train.iloc[:,1]
    log_y_tr = np.log1p(y_tr)
    x_val = val.iloc[:,2:]
    y_val = val.iloc[:,1]
    log_y_val = np.log1p(y_val)
    
    # --- INSERT YOUR MODEL -----
    model = LinearRegression().fit(x_tr, log_y_tr)
    log_y_tr_pred = model.predict(x_tr)
    # ---------------------------
    
    log_y_tr_pred = [0 if i < 0 else i for i in log_y_tr_pred]
    log_y_val_pred = model.predict(x_val)
    log_y_val_pred = [0 if i < 0 else i for i in log_y_val_pred]
    
    mse_tr, mse_val = getMse(x_tr, train, val, log_y_tr_pred, log_y_val_pred)
    train_mse.append(mse_tr)
    train_rmse.append(np.sqrt(mse_tr))
    val_mse.append(mse_val)
    val_rmse.append(np.sqrt(mse_val))


print('\n\nAverage:')
print('train_mse_5fold', np.mean(train_mse))
print('train_rmse_5fold', np.mean(train_rmse))
print('val_mse_5fold', np.mean(val_mse))
print('val_rmse_5fold', np.mean(val_rmse))





fold: 0
train_mse 3.3688616415988126
train_rmse 1.8354458972137568
val_mse 3.3670805144069296
val_rmse 1.834960630206253


fold: 1
train_mse 3.3559453584839813
train_rmse 1.8319239499728097
val_mse 3.412115877499077
val_rmse 1.8471913483716507


fold: 2
train_mse 3.3634080097646457
train_rmse 1.8339596532543037
val_mse 3.3908413702120077
val_rmse 1.841423734563017


fold: 3
train_mse 3.3813119703690826
train_rmse 1.8388344053690866
val_mse 3.317557888857866
val_rmse 1.8214164512427864


fold: 4
train_mse 3.367557241473947
train_rmse 1.835090526778978
val_mse 3.3544963237283114
val_rmse 1.831528411935865


Average:
train_mse_5fold 3.367416844338094
train_rmse_5fold 1.8350508865177872
val_mse_5fold 3.3684183949408384
val_rmse_5fold 1.8353041152639142


In [ ]:
# Testset prediction
model = LinearRegression()
filename = 'linreg'
testset_pred(model, filename)

### Polynomial Regression

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [22]:
train_mse = []
train_rmse = []
val_mse = []
val_rmse = []

for i in range(fold):
    print('\n\nfold:', i)
    val = processed_train_df[processed_train_df['fullVisitorId'].isin(id_cv[i])]
    train = processed_train_df[~processed_train_df['fullVisitorId'].isin(id_cv[i])]
    x_tr = train.iloc[:,2:]
    y_tr = train.iloc[:,1]
    log_y_tr = np.log1p(y_tr)
    x_val = val.iloc[:,2:]
    y_val = val.iloc[:,1]
    log_y_val = np.log1p(y_val)
    
    # --- INSERT YOUR MODEL -----
    model_pipeline = Pipeline([('poly',PolynomialFeatures(degree=2)),
                  ('linear', LinearRegression(fit_intercept=False))])
    model = model_pipeline.fit(x_tr, log_y_tr)
    log_y_tr_pred = model.predict(x_tr)
    # ---------------------------
    
    log_y_tr_pred = [0 if i < 0 else i for i in log_y_tr_pred]
    log_y_val_pred = model.predict(x_val)
    log_y_val_pred = [0 if i < 0 else i for i in log_y_val_pred]
    
    mse_tr, mse_val = getMse(x_tr, train, val, log_y_tr_pred, log_y_val_pred)
    train_mse.append(mse_tr)
    train_rmse.append(np.sqrt(mse_tr))
    val_mse.append(mse_val)
    val_rmse.append(np.sqrt(mse_val))


print('\n\nAverage:')
print('train_mse_5fold', np.mean(train_mse))
print('train_rmse_5fold', np.mean(train_rmse))
print('val_mse_5fold', np.mean(val_mse))
print('val_rmse_5fold', np.mean(val_rmse))






fold: 0
train_mse 2.946886515764455
train_rmse 1.7166497941526848
val_mse 2.9821568880249196
val_rmse 1.7268922630045338


fold: 1
train_mse 2.93871400791534
train_rmse 1.7142677760243117
val_mse 2.9974578863610244
val_rmse 1.7313168070463085


fold: 2
train_mse 2.9456686275685318
train_rmse 1.7162950292908652
val_mse 2.9795638501076303
val_rmse 1.7261413181161125


fold: 3
train_mse 2.9604085429955624
train_rmse 1.7205837797083763
val_mse 2.914674453888298
val_rmse 1.7072417678490348


fold: 4
train_mse 2.9550621375684254
train_rmse 1.71902941730746
val_mse 2.9128125778127876
val_rmse 1.7066963929805405


Average:
train_mse_5fold 2.949347966362463
train_rmse_5fold 1.7173651592967396
val_mse_5fold 2.957333131238932
val_rmse_5fold 1.719657709799306


In [ ]:
# Testset prediction
model = Pipeline([('poly',PolynomialFeatures(degree=2)),
                  ('linear', LinearRegression(fit_intercept=False))])
filename = 'polyreg'
testset_pred(model, filename)

### Regression Tree

In [23]:
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

In [27]:
train_mse = []
train_rmse = []
val_mse = []
val_rmse = []

for i in range(fold):
    print('\n\nfold:', i)
    val = processed_train_df[processed_train_df['fullVisitorId'].isin(id_cv[i])]
    train = processed_train_df[~processed_train_df['fullVisitorId'].isin(id_cv[i])]
    x_tr = train.iloc[:,2:]
    y_tr = train.iloc[:,1]
    log_y_tr = np.log1p(y_tr)
    x_val = val.iloc[:,2:]
    y_val = val.iloc[:,1]
    log_y_val = np.log1p(y_val)
    
    # --- INSERT YOUR MODEL -----
    model = DecisionTreeRegressor(max_depth=10)
    model.fit(x_tr, log_y_tr)
    log_y_tr_pred = model.predict(x_tr)
    # ---------------------------
    
    log_y_tr_pred = [0 if i < 0 else i for i in log_y_tr_pred]
    log_y_val_pred = model.predict(x_val)
    log_y_val_pred = [0 if i < 0 else i for i in log_y_val_pred]
    
    mse_tr, mse_val = getMse(x_tr, train, val, log_y_tr_pred, log_y_val_pred)
    train_mse.append(mse_tr)
    train_rmse.append(np.sqrt(mse_tr))
    val_mse.append(mse_val)
    val_rmse.append(np.sqrt(mse_val))


print('\n\nAverage:')
print('train_mse_5fold', np.mean(train_mse))
print('train_rmse_5fold', np.mean(train_rmse))
print('val_mse_5fold', np.mean(val_mse))
print('val_rmse_5fold', np.mean(val_rmse))



fold: 0
train_mse 2.49313632902639
train_rmse 1.5789668549486369
val_mse 2.889406238725187
val_rmse 1.6998253553601284


fold: 1
train_mse 2.4808600973933994
train_rmse 1.5750746323248939
val_mse 2.9277044337919076
val_rmse 1.711053603424483


fold: 2
train_mse 2.4996003245842373
train_rmse 1.5810124365684912
val_mse 2.8806438875379197
val_rmse 1.6972459714307528


fold: 3
train_mse 2.4929157881153308
train_rmse 1.5788970163108582
val_mse 2.8183075362512158
val_rmse 1.6787815629947858


fold: 4
train_mse 2.492124668011695
train_rmse 1.5786464670760503
val_mse 2.857506791920797
val_rmse 1.6904161593882132


Average:
train_mse_5fold 2.4917274414262107
train_rmse_5fold 1.578519481445786
val_mse_5fold 2.874713777645405
val_rmse_5fold 1.6954645305196727


In [28]:
# Testset prediction
model = DecisionTreeRegressor(max_depth=10)
filename = 'regtree'
testset_pred(model, filename)

(617242, 2)
